In [1]:
!pip install transformers==2.8.0

     |████████████████████████████████| 573kB 2.8MB/s 
     |████████████████████████████████| 3.7MB 13.7MB/s 
     |████████████████████████████████| 890kB 31.7MB/s 
     |████████████████████████████████| 1.1MB 46.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=349404a23dc2acdb0e5be73ade5a91cab62ab279d8e661048f108d936e3d5694
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


# Config

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import torch
import transformers
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = 'large'
nlayer = 12 if config == 'base' else 24
nsamples = 5000 if config == 'base' else 10000

model = transformers.BertForMaskedLM.from_pretrained('bert-'+config+'-uncased', output_hidden_states=True).to(device)
tokenizer = transformers.BertTokenizer.from_pretrained('bert-'+config+'-uncased')
# turn on eval mode
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, elementw

# WEAT

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def swAB(W, A, B):
    """Calculates differential cosine-similarity between word vectors in W, A and W, B
        Arguments
                W, A, B : n x d matrix of word embeddings stored row wise
    """
    WA = cosine_similarity(W,A)
    WB = cosine_similarity(W,B)
    
    #Take mean along columns
    WAmean = np.mean(WA, axis = 1)
    WBmean = np.mean(WB, axis = 1)
    
    return (WAmean - WBmean)
  
def test_statistic(X, Y, A, B):
    """Calculates test-statistic between the pair of association words and target words
        Arguments
                X, Y, A, B : n x d matrix of word embeddings stored row wise
        Returns
                Test Statistic
    """
    return (sum(swAB(X, A, B)) - sum(swAB(Y, A, B)))

def weat_effect_size(X, Y, A, B, embd):
    """Computes the effect size for the given list of association and target word pairs
        Arguments
                X, Y : List of association words
                A, B : List of target words
                embd : Dictonary of word-to-embedding for all words
        Returns
                Effect Size
    """
    Xmat = np.array([embd[w] for w in X if w in embd])
    Ymat = np.array([embd[w] for w in Y if w in embd])
    Amat = np.array([embd[w] for w in A if w in embd])
    Bmat = np.array([embd[w] for w in B if w in embd])
    XuY = list(set(X).union(Y))
    XuYmat = []
    for w in XuY:
        if w.lower() in embd:
            XuYmat.append(embd[w.lower()])
    XuYmat = np.array(XuYmat)
    d = (np.mean(swAB(Xmat,Amat,Bmat)) - np.mean(swAB(Ymat,Amat,Bmat)))/np.std(swAB(XuYmat, Amat, Bmat))
    return d

def random_permutation(iterable, r=None):
    """Returns a random permutation for any iterable object"""
    pool = tuple(iterable)
    r = len(pool) if r is None else r
    return tuple(random.sample(pool, r))

def weat_p_value(X, Y, A, B, embd, sample=None):
    np.random.seed(42)
    random.seed(42)
    """Computes the one-sided P value for the given list of association and target word pairs
        Arguments
                X, Y : List of association words
                A, B : List of target words
                embd : Dictonary of word-to-embedding for all words
                sample : Number of random permutations used.
        Returns
    """
    size_of_permutation = min(len(X), len(Y))
    X_Y = X + Y
    test_stats_over_permutation = []
    
    Xmat = np.array([embd[w.lower()] for w in X if w.lower() in embd])
    Ymat = np.array([embd[w.lower()] for w in Y if w.lower() in embd])
    Amat = np.array([embd[w.lower()] for w in A if w.lower() in embd])
    Bmat = np.array([embd[w.lower()] for w in B if w.lower() in embd])
    
    if not sample:
        permutations = combinations(X_Y, size_of_permutation)
    else:
        permutations = [random_permutation(X_Y, size_of_permutation) for s in range(sample)]
        
    for Xi in permutations:
        Yi = filterfalse(lambda w:w in Xi, X_Y)
        Ximat = np.array([embd[w.lower()] for w in Xi if w.lower() in embd])
        Yimat = np.array([embd[w.lower()] for w in Yi if w.lower() in embd])
        test_stats_over_permutation.append(test_statistic(Ximat, Yimat, Amat, Bmat))
        
    unperturbed = test_statistic(Xmat, Ymat, Amat, Bmat)
    
    is_over = np.array([o > unperturbed for o in test_stats_over_permutation])
    
    return is_over.sum() / is_over.size

In [ ]:
def category(type=1):
    m_words = "male, man, boy, brother, he, him, his, son, father, uncle, grandfather".lower().replace(" ", "").split(',')
    f_words = "female, woman, girl, sister, she, her, hers, daughter, mother, aunt, grandmother".lower().replace(" ", "").split(',')
    if type == 1:
        #B1: career vs family
        x_words = "executive, management, professional, corporation, salary, office, business, career".lower().replace(" ", "").split(',')
        y_words = "home, parents, children, family, cousins, marriage, wedding, relatives".lower().replace(" ", "").split(',')
    elif type == 2:
        #B2: maths vs arts
        x_words = "math, algebra, geometry, calculus, equations, computation, numbers, addition".lower().replace(" ", "").split(',')
        y_words = "poetry, art, Shakespeare, dance, literature, novel, symphony, drama".lower().replace(" ", "").split(',')
    elif type == 3:
        #B3: science vs arts
        x_words = "science, technology, physics, chemistry, Einstein, NASA, experiment, astronomy".lower().replace(" ", "").split(',')
        y_words = "poetry, art, Shakespeare, dance, literature, novel, symphony, drama".lower().replace(" ", "").split(',')
    elif type == 4:
        #B4: intelligence vs appearance
        x_words = "precocious, resourceful, inquisitive, genius, inventive, astute, adaptable, reflective,discerning, intuitive, inquiring, judicious, analytical, apt, venerable, imaginative,shrewd, thoughtful, wise, smart, ingenious, clever, brilliant, logical, intelligent".lower().replace(" ", "").split(',')
        y_words = "alluring, voluptuous, blushing, homely, plump, sensual, gorgeous, slim, bald,athletic, fashionable, stout, ugly, muscular, slender, feeble, handsome, healthy,attractive, fat, weak, thin, pretty, beautiful, strong".lower().replace(" ", "").split(',')
    elif type == 5:
        #B5: strength vs weakness
        x_words = "power, strong, confident, dominant, potent, command, assert, loud, bold, succeed,triumph, leader, shout, dynamic, winner".lower().replace(" ", "").split(',')
        y_words = "weak, surrender, timid, vulnerable, weakness, wispy, withdraw, yield, failure, shy,follow, lose, fragile, afraid, loser".lower().replace(" ", "").split(',')
    return m_words, f_words, x_words, y_words

def category_2(type=1):
    if type == 1:
        #Career/Family
        x_words = 'John, Paul, Mike, Kevin, Steve, Greg, Jeff, Bill'.lower().replace(" ", "").split(',')
        y_words = 'Amy, Joan, Lisa, Sarah, Diana, Kate, Ann, Donna'.lower().replace(" ", "").split(',')
        m_words = 'executive, management, professional, corporation, salary, office, business, career'.lower().replace(" ", "").split(',')
        f_words = 'home, parents, children, family, cousins, marriage, wedding, relatives'.lower().replace(" ", "").split(',')
    elif type == 2:
        #Math/Art
        x_words = 'math, algebra, geometry, calculus, equations, computation, numbers, addition'.lower().replace(" ", "").split(',')
        y_words = 'poetry, art, dance, literature, novel, symphony, drama, sculpture'.lower().replace(" ", "").split(',')
        m_words = 'male, man, boy, brother, he, him, his, son'.lower().replace(" ", "").split(',')
        f_words = 'female, woman, girl, sister, she, her, hers, daughter'.lower().replace(" ", "").split(',')
    elif type == 3:
        #Science/Art
        x_words = 'science, technology, physics, chemistry, Einstein, NASA, experiment, astronomy'.lower().replace(" ", "").split(',')
        y_words = 'poetry, art, Shakespeare, dance, literature, novel, symphony, drama'.lower().replace(" ", "").split(',')
        m_words = 'brother, father, uncle, grandfather, son, he, his, him'.lower().replace(" ", "").split(',')
        f_words = '  , mother, aunt, grandmother, daughter, she, hers, her'.lower().replace(" ", "").split(',')
    return m_words, f_words, x_words, y_words

In [ ]:
import numpy as np
from itertools import combinations, filterfalse
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.keyedvectors import KeyedVectors
import pandas as pd
import random
import sys
import os
import pickle
random.seed(42)

In [ ]:
!find /content/ -name '*eigvecs*' | xargs  rm -rf

In [ ]:
def get_bert_embedding(model, wordlist, is_targets=1):
    vecss = torch.Tensor().to(device)
    for w in wordlist:
        text = w + ' is ' + tokenizer.mask_token + '.' if not is_targets else tokenizer.mask_token + ' is ' + w + '.'
        vec = tokenizer.prepare_for_model(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text)),
                                            return_token_type_ids=False, return_tensors='pt')['input_ids'].to(device)
        vecs = vec.clone().detach()
        # get output
        vecs = model.bert(vecs)[0]#[2][nlayer]
        vecs = vecs[0][1:-4,:].mean(dim=0).unsqueeze(0) if not is_targets else vecs[0][3:-2,:].mean(dim=0).unsqueeze(0)
        vecss = torch.cat((vecss,vecs))
    return vecss

import densray_bert as bbert
def get_eigvecs_dict(layer=-1):
    eigvecs_dict = {}
    #-1:apply to all layers
    if layer==-1:
        #'/content/drive/My Drive/negc'+config+'_'+str(nsamples)+'_'+str(l)+'.pt'
        #'/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(l)+'.pt'
        for l in range(nlayer):
            eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(l)+'.pt', True)
    elif layer ==-2:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(l)+'.pt', False)
    else:
        for l in range(nlayer):
            if l==layer:
                eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(l)+'.pt', True)
            else:
                eigvecs_dict[str(l)] = ('/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(l)+'.pt', False)
    return eigvecs_dict

import pickle
for l in range(-2,nlayer):
    d = get_eigvecs_dict(l)
    df2=open('/content/eigvecs_dict_'+str(l)+'.txt','wb')
    pickle.dump(d,df2)
    df2.close()

def eval_per_layer(layer=-2):
    config_class = get_eigvecs_dict(layer)
    model = bbert.BertForMaskedLM_1.from_pretrained('bert-'+config+'-uncased', eigvecs_dict=get_eigvecs_dict(l)).to(device)
    # turn on eval mode
    model.eval()
    m = get_bert_embedding(model, m_words, is_targets=0).cpu().detach().numpy()
    f = get_bert_embedding(model, f_words, is_targets=0).cpu().detach().numpy()
    x = get_bert_embedding(model, x_words, is_targets=1).cpu().detach().numpy()
    y = get_bert_embedding(model, y_words, is_targets=1).cpu().detach().numpy()
    embed = {}
    for i in range(len(m_words)): embed[m_words[i]] = m[i]
    for i in range(len(f_words)): embed[f_words[i]] = f[i]
    for i in range(len(x_words)): embed[x_words[i]] = x[i]
    for i in range(len(y_words)): embed[y_words[i]] = y[i]
    return embed



for t in range(1,4):
    m_words, f_words, x_words, y_words = category_2(t)
    l=-2
    # no densray
    embed = eval_per_layer(layer=l)
    d = weat_effect_size(x_words, y_words, m_words, f_words, embed)
    p = weat_p_value(x_words, y_words, m_words, f_words, embed, sample=1000)
    print(round(d,4),round(p,4),)
    #densray
    for l in range(-1, 0):
        # densray
        embed = eval_per_layer(layer=l)
        d_densray =  weat_effect_size(x_words, y_words, m_words, f_words, embed)
        p_densray = weat_p_value(x_words, y_words, m_words, f_words, embed, sample=1000)
        print(round(d_densray,4), round(p_densray,4))
    print('\n')

0.3878 0.237
0.4015 0.242


1.2742 0.004
0.3707 0.229


0.7759 0.06
0.0786 0.439




In [ ]:
!find /content/ -name '*eigvecs*' | xargs  rm -rf

In [ ]:
def get_bert_embedding(model, wordlist, is_targets=1):
    vecss = torch.Tensor().to(device)
    for w in wordlist:
        text = w + ' is ' + tokenizer.mask_token + '.' if not is_targets else tokenizer.mask_token + ' is ' + w + '.'
        vec = tokenizer.prepare_for_model(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text)),
                                            return_token_type_ids=False, return_tensors='pt')['input_ids'].to(device)
        vecs = vec.clone().detach()
        # get output
        vecs = model.bert(vecs)[0]#[2][nlayer]
        vecs = vecs[0][1:-4,:].mean(dim=0).unsqueeze(0) if not is_targets else vecs[0][3:-2,:].mean(dim=0).unsqueeze(0)
        vecss = torch.cat((vecss,vecs))
    return vecss

import hard_bert as bbert
def get_eigvecs_dict(layer=-1):
    eigvecs_dict = {}
    #-1:apply to all layers
    if layer==-1:
        #'/content/drive/My Drive/negc'+config+'_'+str(nsamples)+'_'+str(l)+'.pt'
        #'/content/drive/My Drive/eigvecs_'+config+'_new_'+str(nsamples)+'_'+str(l)+'.pt'
        for l in range(nlayer):
            eigvecs_dict[str(l)] = ('/content/drive/My Drive/pc1'+config+'_'+str(nsamples)+'_'+str(l)+'.pt', True)
    elif layer ==-2:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = ('/content/drive/My Drive/pc1'+config+'_'+str(nsamples)+'_'+str(l)+'.pt', False)
    else:
        for l in range(nlayer):
            if l==layer:
                eigvecs_dict[str(l)] = ('/content/drive/My Drive/pc1'+config+'_'+str(nsamples)+'_'+str(l)+'.pt', True)
            else:
                eigvecs_dict[str(l)] = ('/content/drive/My Drive/pc1'+config+'_'+str(nsamples)+'_'+str(l)+'.pt', False)
    return eigvecs_dict

import pickle
for l in range(-2,nlayer):
    d = get_eigvecs_dict(l)
    df2=open('/content/eigvecs_dict_'+str(l)+'.txt','wb')
    pickle.dump(d,df2)
    df2.close()

def eval_per_layer(layer=-2):
    config_class = get_eigvecs_dict(layer)
    model = bbert.BertForMaskedLM_1.from_pretrained('bert-'+config+'-uncased', eigvecs_dict=get_eigvecs_dict(l)).to(device)
    # turn on eval mode
    model.eval()
    m = get_bert_embedding(model, m_words, is_targets=0).cpu().detach().numpy()
    f = get_bert_embedding(model, f_words, is_targets=0).cpu().detach().numpy()
    x = get_bert_embedding(model, x_words, is_targets=1).cpu().detach().numpy()
    y = get_bert_embedding(model, y_words, is_targets=1).cpu().detach().numpy()
    embed = {}
    for i in range(len(m_words)): embed[m_words[i]] = m[i]
    for i in range(len(f_words)): embed[f_words[i]] = f[i]
    for i in range(len(x_words)): embed[x_words[i]] = x[i]
    for i in range(len(y_words)): embed[y_words[i]] = y[i]
    return embed



for t in range(1,4):
    m_words, f_words, x_words, y_words = category_2(t)
    l=-2
    # no densray
    embed = eval_per_layer(layer=l)
    d = weat_effect_size(x_words, y_words, m_words, f_words, embed)
    p = weat_p_value(x_words, y_words, m_words, f_words, embed, sample=1000)
    print(round(d,4),round(p,4),)
    #densray
    for l in range(-1, 0):
        # densray
        embed = eval_per_layer(layer=l)
        d_densray =  weat_effect_size(x_words, y_words, m_words, f_words, embed)
        p_densray = weat_p_value(x_words, y_words, m_words, f_words, embed, sample=1000)
        print(round(d_densray,4), round(p_densray,4))
    print('\n')

0.3878 0.237
0.1482 0.391


1.2742 0.004
-0.1995 0.642


0.7759 0.06
-0.6103 0.865




# SEAT

In [5]:
!wget https://raw.githubusercontent.com/W4ngatang/sent-bias/master/tests/sent-weat6.jsonl
!wget https://raw.githubusercontent.com/W4ngatang/sent-bias/master/tests/sent-weat7.jsonl
!wget https://raw.githubusercontent.com/W4ngatang/sent-bias/master/tests/sent-weat8.jsonl

--2020-07-01 06:20:02--  https://raw.githubusercontent.com/W4ngatang/sent-bias/master/tests/sent-weat6.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9880 (9.6K) [text/plain]
Saving to: ‘sent-weat6.jsonl’

sent-weat6.jsonl    100%[===================>]   9.65K  --.-KB/s    in 0s      

2020-07-01 06:20:02 (55.9 MB/s) - ‘sent-weat6.jsonl’ saved [9880/9880]

--2020-07-01 06:20:03--  https://raw.githubusercontent.com/W4ngatang/sent-bias/master/tests/sent-weat7.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8703 (8.5K) [text/plain

In [6]:
import json
import numpy as np

WEAT_SETS = ["targ1", "targ2", "attr1", "attr2"]
CATEGORY = "category"

def load_json(sent_file):
    all_data = json.load(open(sent_file, 'r'))
    data = {}
    for k, v in all_data.items():
        examples = v["examples"]
        data[k] = examples
        v["examples"] = examples
    return all_data  # data

In [7]:
''' Implements the WEAT tests '''
import math
import itertools as it
import numpy as np
import scipy.special
import scipy.stats

# X and Y are two sets of target words of equal size.
# A and B are two sets of attribute words.


def cossim(x, y):
    return np.dot(x, y) / math.sqrt(np.dot(x, x) * np.dot(y, y))


def construct_cossim_lookup(XY, AB):
    cossims = np.zeros((len(XY), len(AB)))
    for xy in XY:
        for ab in AB:
            cossims[xy, ab] = cossim(XY[xy], AB[ab])
    return cossims


def s_wAB(A, B, cossims):
    return cossims[:, A].mean(axis=1) - cossims[:, B].mean(axis=1)


def s_XAB(X, s_wAB_memo):
    return s_wAB_memo[X].sum()


def s_XYAB(X, Y, s_wAB_memo):
    return s_XAB(X, s_wAB_memo) - s_XAB(Y, s_wAB_memo)


def p_val_permutation_test(X, Y, A, B, n_samples, cossims, parametric=False):
    X = np.array(list(X), dtype=np.int)
    Y = np.array(list(Y), dtype=np.int)
    A = np.array(list(A), dtype=np.int)
    B = np.array(list(B), dtype=np.int)

    assert len(X) == len(Y)
    size = len(X)
    s_wAB_memo = s_wAB(A, B, cossims=cossims)
    XY = np.concatenate((X, Y))

    if parametric:
        s = s_XYAB(X, Y, s_wAB_memo)
        samples = []
        for _ in range(n_samples):
            np.random.shuffle(XY)
            Xi = XY[:size]
            Yi = XY[size:]
            assert len(Xi) == len(Yi)
            si = s_XYAB(Xi, Yi, s_wAB_memo)
            samples.append(si)

        # Compute sample standard deviation and compute p-value by
        # assuming normality of null distribution
        (shapiro_test_stat, shapiro_p_val) = scipy.stats.shapiro(samples)
        sample_mean = np.mean(samples)
        sample_std = np.std(samples, ddof=1)
        p_val = scipy.stats.norm.sf(s, loc=sample_mean, scale=sample_std)
        return p_val

    else:
        s = s_XAB(X, s_wAB_memo)
        total_true = 0
        total_equal = 0
        total = 0

        num_partitions = int(scipy.special.binom(2 * len(X), len(X)))
        if num_partitions > n_samples:
            # We only have as much precision as the number of samples drawn;
            # bias the p-value (hallucinate a positive observation) to
            # reflect that.
            total_true += 1
            total += 1
            for _ in range(n_samples - 1):
                np.random.shuffle(XY)
                Xi = XY[:size]
                assert 2 * len(Xi) == len(XY)
                si = s_XAB(Xi, s_wAB_memo)
                if si > s:
                    total_true += 1
                elif si == s:  # use conservative test
                    total_true += 1
                    total_equal += 1
                total += 1
        else:
            for Xi in it.combinations(XY, len(X)):
                Xi = np.array(Xi, dtype=np.int)
                assert 2 * len(Xi) == len(XY)
                si = s_XAB(Xi, s_wAB_memo)
                if si > s:
                    total_true += 1
                elif si == s:  # use conservative test
                    total_true += 1
                    total_equal += 1
                total += 1
        return total_true / total


def mean_s_wAB(X, A, B, cossims):
    return np.mean(s_wAB(A, B, cossims[X]))


def stdev_s_wAB(X, A, B, cossims):
    return np.std(s_wAB(A, B, cossims[X]), ddof=1)


def effect_size(X, Y, A, B, cossims):
    X = list(X)
    Y = list(Y)
    A = list(A)
    B = list(B)

    numerator = mean_s_wAB(X, A, B, cossims=cossims) - mean_s_wAB(Y, A, B, cossims=cossims)
    denominator = stdev_s_wAB(X + Y, A, B, cossims=cossims)
    return numerator / denominator


def convert_keys_to_ints(X, Y):
    return (
        dict((i, v) for (i, (k, v)) in enumerate(X.items())),
        dict((i + len(X), v) for (i, (k, v)) in enumerate(Y.items())),
    )


def run_test(encs, n_samples, parametric=False):
    X, Y = encs["targ1"]["encs"], encs["targ2"]["encs"]
    A, B = encs["attr1"]["encs"], encs["attr2"]["encs"]

    # First convert all keys to ints to facilitate array lookups
    (X, Y) = convert_keys_to_ints(X, Y)
    (A, B) = convert_keys_to_ints(A, B)
    XY = X.copy()
    XY.update(Y)
    AB = A.copy()
    AB.update(B)
    cossims = construct_cossim_lookup(XY, AB)
    pval = p_val_permutation_test(X, Y, A, B, n_samples, cossims=cossims, parametric=parametric)
    esize = effect_size(X, Y, A, B, cossims=cossims)
    return esize, pval

In [8]:
method = 'densray' #hard #conceptor
if method == 'densray':
    import densray_bert as bbert
    path = '/content/drive/My Drive/eigvecs_'+config+'_new_'
elif method == 'hard':
    import hard_bert as bbert
    path = '/content/drive/My Drive/pc1'+config+'_'
elif method == 'conceptor':
    import conceptor_bert as bbert
    path = '/content/drive/My Drive/negc'+config+'_'

def get_bert_embedding(model, seat_dict):
    for weat in WEAT_SETS:
        wordlist = seat_dict[weat]['examples']
        vecss = {}
        for w in wordlist:
            vec = tokenizer.prepare_for_model(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(w.lower())),
                                              return_token_type_ids=False, return_tensors='pt')['input_ids'].to(device)
            vecs = vec.clone().detach()
            # get embedding of [CLS] as aggregation of the sentence
            vecs = model.bert(vecs)[0][0][0]
            vecss[w] = vecs.cpu().detach().numpy()
        seat_dict[weat]['encs'] = vecss  
    return seat_dict

def get_eigvecs_dict(layer=-1):
    eigvecs_dict = {}
    #-1:apply to all layers, -2 nothing, l only on layer lth.
    if layer==-1:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', True)
    elif layer ==-2:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', False)
    else:
        for l in range(nlayer):
            if l==layer:
                eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', True)
            else:
                eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', False)
    return eigvecs_dict

def eval_per_layer(seat_dict, layer=-2):
    model = bbert.BertForMaskedLM_1.from_pretrained('bert-'+config+'-uncased', eigvecs_dict=get_eigvecs_dict(layer)).to(device)
    #model = transformers.BertForMaskedLM.from_pretrained('bert-'+config+'-uncased').to(device)
    # turn on eval mode
    model.eval()
    seat_dict = get_bert_embedding(model, seat_dict)#.cpu().detach().numpy()
    return seat_dict



for c in range(6,9):
    seat_dict = load_json('/content/sent-weat'+str(c)+'.jsonl')
    l=-2
    # no densray
    seat_dict = eval_per_layer(seat_dict, layer=l)
    d, p = run_test(seat_dict, n_samples=5000, parametric=True)
    print(round(d,4),round(p,4),)
    #densray
    for l in range(-1, 0):
        # densray
        seat_dict = eval_per_layer(seat_dict, layer=l)
        d_densray, p_densray = run_test(seat_dict, n_samples=5000, parametric=True)
        print(round(d_densray,4), round(p_densray,4))
    print('\n')

0.4997 0.0021
0.1349 0.2156


0.3834 0.0105
-0.743 1.0


-0.3033 0.9463
-0.6633 0.9998




In [9]:
method = 'hard' #hard #conceptor
if method == 'densray':
    import densray_bert as bbert
    path = '/content/drive/My Drive/eigvecs_'+config+'_new_'
elif method == 'hard':
    import hard_bert as bbert
    path = '/content/drive/My Drive/pc1'+config+'_'
elif method == 'conceptor':
    import conceptor_bert as bbert
    path = '/content/drive/My Drive/negc'+config+'_'

def get_bert_embedding(model, seat_dict):
    for weat in WEAT_SETS:
        wordlist = seat_dict[weat]['examples']
        vecss = {}
        for w in wordlist:
            vec = tokenizer.prepare_for_model(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(w.lower())),
                                              return_token_type_ids=False, return_tensors='pt')['input_ids'].to(device)
            vecs = vec.clone().detach()
            # get embedding of [CLS] as aggregation of the sentence
            vecs = model.bert(vecs)[0][0][0]
            vecss[w] = vecs.cpu().detach().numpy()
        seat_dict[weat]['encs'] = vecss  
    return seat_dict

def get_eigvecs_dict(layer=-1):
    eigvecs_dict = {}
    #-1:apply to all layers, -2 nothing, l only on layer lth.
    if layer==-1:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', True)
    elif layer ==-2:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', False)
    else:
        for l in range(nlayer):
            if l==layer:
                eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', True)
            else:
                eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', False)
    return eigvecs_dict

def eval_per_layer(seat_dict, layer=-2):
    model = bbert.BertForMaskedLM_1.from_pretrained('bert-'+config+'-uncased', eigvecs_dict=get_eigvecs_dict(layer)).to(device)
    #model = transformers.BertForMaskedLM.from_pretrained('bert-'+config+'-uncased').to(device)
    # turn on eval mode
    model.eval()
    seat_dict = get_bert_embedding(model, seat_dict)#.cpu().detach().numpy()
    return seat_dict



for c in range(6,9):
    seat_dict = load_json('/content/sent-weat'+str(c)+'.jsonl')
    l=-2
    # no densray
    seat_dict = eval_per_layer(seat_dict, layer=l)
    d, p = run_test(seat_dict, n_samples=5000, parametric=True)
    print(round(d,4),round(p,4),)
    #densray
    for l in range(-1, 0):
        # densray
        seat_dict = eval_per_layer(seat_dict, layer=l)
        d_densray, p_densray = run_test(seat_dict, n_samples=5000, parametric=True)
        print(round(d_densray,4), round(p_densray,4))
    print('\n')

0.4997 0.0023
0.0664 0.3518


0.3834 0.0114
0.3839 0.0096


-0.3033 0.9475
-0.0303 0.559




In [10]:
method = 'conceptor' #hard #conceptor
if method == 'densray':
    import densray_bert as bbert
    path = '/content/drive/My Drive/eigvecs_'+config+'_new_'
elif method == 'hard':
    import hard_bert as bbert
    path = '/content/drive/My Drive/pc1'+config+'_'
elif method == 'conceptor':
    import conceptor_bert as bbert
    path = '/content/drive/My Drive/negc'+config+'_'

def get_bert_embedding(model, seat_dict):
    for weat in WEAT_SETS:
        wordlist = seat_dict[weat]['examples']
        vecss = {}
        for w in wordlist:
            vec = tokenizer.prepare_for_model(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(w.lower())),
                                              return_token_type_ids=False, return_tensors='pt')['input_ids'].to(device)
            vecs = vec.clone().detach()
            # get embedding of [CLS] as aggregation of the sentence
            vecs = model.bert(vecs)[0][0][0]
            vecss[w] = vecs.cpu().detach().numpy()
        seat_dict[weat]['encs'] = vecss  
    return seat_dict

def get_eigvecs_dict(layer=-1):
    eigvecs_dict = {}
    #-1:apply to all layers, -2 nothing, l only on layer lth.
    if layer==-1:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', True)
    elif layer ==-2:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', False)
    else:
        for l in range(nlayer):
            if l==layer:
                eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', True)
            else:
                eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', False)
    return eigvecs_dict

def eval_per_layer(seat_dict, layer=-2):
    model = bbert.BertForMaskedLM_1.from_pretrained('bert-'+config+'-uncased', eigvecs_dict=get_eigvecs_dict(layer)).to(device)
    #model = transformers.BertForMaskedLM.from_pretrained('bert-'+config+'-uncased').to(device)
    # turn on eval mode
    model.eval()
    seat_dict = get_bert_embedding(model, seat_dict)#.cpu().detach().numpy()
    return seat_dict



for c in range(6,9):
    seat_dict = load_json('/content/sent-weat'+str(c)+'.jsonl')
    l=-2
    # no densray
    seat_dict = eval_per_layer(seat_dict, layer=l)
    d, p = run_test(seat_dict, n_samples=5000, parametric=True)
    print(round(d,4),round(p,4),)
    #densray
    for l in range(11, 12):
        # densray
        seat_dict = eval_per_layer(seat_dict, layer=l)
        d_densray, p_densray = run_test(seat_dict, n_samples=5000, parametric=True)
        print(round(d_densray,4), round(p_densray,4))
    print('\n')

0.4997 0.0021
0.0554 0.3737


0.3834 0.0103
-0.6035 0.9999


-0.3033 0.9475
-0.2987 0.9424




In [ ]:
seat_dict["targ1"]["encs"]['This is John.'].shape

(768,)

In [ ]:
1.0436 0.0
-0.0832 0.6804
0.3613 0.0205

0.1761 0.1465
-0.0643 0.652
-0.4707 0.9976

0.8134 0.0
-0.1044 0.7075
0.4175 0.0128










0.1078 0.2498
-0.4781 0.9988
-0.1085 0.754

0.7247 0.0
-0.0976 0.7211
0.1075 0.26

1.0039 0.0
0.3613 0.027
0.7547 0.0

# CEAT

In [ ]:
#weat  6
male = ['John', 'Paul', 'Mike', 'Kevin', 'Steve', 'Greg', 'Jeff', 'Bill']
female = ['Amy', 'Joan', 'Lisa', 'Sarah', 'Diana', 'Kate', 'Ann', 'Donna']
career = ['executive', 'management', 'professional', 'corporation', 'salary', 'office', 'business', 'career']
family = ['home', 'parents', 'children', 'family', 'cousins', 'marriage', 'wedding', 'relatives']

#weat 7
maths = ['math', 'algebra', 'geometry', 'calculus', 'equations', 'computation', 'numbers', 'addition']
arts = ['poetry', 'art', 'dance', 'literature', 'novel', 'symphony', 'drama', 'sculpture']
male_term = ['male', 'man', 'boy', 'brother', 'he', 'him', 'his', 'son']
female_term = ['female', 'woman', 'girl', 'sister', 'she', 'her', 'hers', 'daughter']

#weat 8
science = ['science', 'technology', 'physics', 'chemistry', 'Einstein', 'NASA', 'experiment', 'astronomy']
arts_8 = ['poetry', 'art', 'Shakespeare', 'dance', 'literature', 'novel', 'symphony', 'drama']
male_term_8 = ['brother', 'father', 'uncle', 'grandfather', 'son', 'he', 'his', 'him']
female_term_8 = ['sister', 'mother', 'aunt', 'grandmother', 'daughter', 'she', 'hers', 'her']

w_all = [i.lower() for i in list(set(male+female+career+family+maths+arts+male_term+female_term+science+arts_8+male_term_8+female_term_8))]

In [ ]:
import pickle
f = open('/content/drive/My Drive/ceat_list_5.pkl', 'rb')
ceat = pickle.load(f)

In [ ]:
method = 'densray' #hard #conceptor
if method == 'densray':
    import densray_bert as bbert
    path = '/content/drive/My Drive/eigvecs_'+config+'_new_'
elif method == 'hard':
    import hard_bert as bbert
    path = '/content/drive/My Drive/pc1'+config+'_'
elif method == 'conceptor':
    import conceptor_bert as bbert
    path = '/content/drive/My Drive/negc'+config+'_'


def get_eigvecs_dict(layer=-1):
    eigvecs_dict = {}
    #-1:apply to all layers, -2 nothing, l only on layer lth.
    if layer==-1:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', True)
    elif layer ==-2:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', False)
    else:
        for l in range(nlayer):
            if l==layer:
                eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', True)
            else:
                eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', False)
    return eigvecs_dict


def for_ceat(in_path, out_path, occ=5):
	#import sss as bert
	#model = bbert.BertForMaskedLM_1.from_pretrained('bert-'+config+'-uncased', eigvecs_dict=get_eigvecs_dict(-1)).to(device)
    # turn on eval mode
	#model.eval()
	#weat  6
	male = ['John', 'Paul', 'Mike', 'Kevin', 'Steve', 'Greg', 'Jeff', 'Bill']
	female = ['Amy', 'Joan', 'Lisa', 'Sarah', 'Diana', 'Kate', 'Ann', 'Donna']
	career = ['executive', 'management', 'professional', 'corporation', 'salary', 'office', 'business', 'career']
	family = ['home', 'parents', 'children', 'family', 'cousins', 'marriage', 'wedding', 'relatives']

	#weat 7
	math = ['math', 'algebra', 'geometry', 'calculus', 'equations', 'computation', 'numbers', 'addition']
	arts = ['poetry', 'art', 'dance', 'literature', 'novel', 'symphony', 'drama', 'sculpture']
	male_term = ['male', 'man', 'boy', 'brother', 'he', 'him', 'his', 'son']
	female_term = ['female', 'woman', 'girl', 'sister', 'she', 'her', 'hers', 'daughter']

	#weat 8
	science = ['science', 'technology', 'physics', 'chemistry', 'Einstein', 'NASA', 'experiment', 'astronomy']
	arts_8 = ['poetry', 'art', 'Shakespeare', 'dance', 'literature', 'novel', 'symphony', 'drama']
	male_term_8 = ['brother', 'father', 'uncle', 'grandfather', 'son', 'he', 'his', 'him']
	female_term_8 = ['sister', 'mother', 'aunt', 'grandmother', 'daughter', 'she', 'hers', 'her']
	w_all = [i.lower() for i in list(set(male+female+career+family+math+arts+male_term+female_term+science+arts_8+male_term_8+female_term_8))]

	dic = {}
	for word in w_all:
		dic[word] = []
		for line in open(in_path):
			line = tokenizer.tokenize(line.strip().lower())[:510]
			if word in line:
				index = line.index(word)
				line = tokenizer.convert_tokens_to_ids(line)
				line = tokenizer.prepare_for_model(line, return_token_type_ids=False, return_tensors='pt')['input_ids'].to(device)
				#get output embedding
				#vec = model(line)[0][0][1:-1][index].cpu().detach().numpy()
				dic[word].append(line)
			if len(dic[word])>=occ:
				break
	f = open(out_path,'wb')
	pickle.dump(dic,f)
	f.close()
 

for_ceat('/content/drive/My Drive/wikien_senttok.txt','/content/drive/My Drive/ceat_list_100.pkl',100)

In [ ]:
''' Implements the WEAT tests '''
import math
import itertools as it
import numpy as np
import scipy.special
import scipy.stats

# X and Y are two sets of target words of equal size.
# A and B are two sets of attribute words.


def cossim(x, y):
    return np.dot(x, y) / math.sqrt(np.dot(x, x) * np.dot(y, y))


def construct_cossim_lookup(XY, AB):
    cossims = np.zeros((len(XY), len(AB)))
    for xy in XY:
        for ab in AB:
            cossims[xy, ab] = cossim(XY[xy], AB[ab])
    return cossims


def s_wAB(A, B, cossims):
    return cossims[:, A].mean(axis=1) - cossims[:, B].mean(axis=1)


def s_XAB(X, s_wAB_memo):
    return s_wAB_memo[X].sum()


def s_XYAB(X, Y, s_wAB_memo):
    return s_XAB(X, s_wAB_memo) - s_XAB(Y, s_wAB_memo)


def p_val_permutation_test(X, Y, A, B, n_samples, cossims, parametric=False):
    X = np.array(list(X), dtype=np.int)
    Y = np.array(list(Y), dtype=np.int)
    A = np.array(list(A), dtype=np.int)
    B = np.array(list(B), dtype=np.int)

    assert len(X) == len(Y)
    size = len(X)
    s_wAB_memo = s_wAB(A, B, cossims=cossims)
    XY = np.concatenate((X, Y))

    if parametric:
        s = s_XYAB(X, Y, s_wAB_memo)
        samples = []
        for _ in range(n_samples):
            np.random.shuffle(XY)
            Xi = XY[:size]
            Yi = XY[size:]
            assert len(Xi) == len(Yi)
            si = s_XYAB(Xi, Yi, s_wAB_memo)
            samples.append(si)

        # Compute sample standard deviation and compute p-value by
        # assuming normality of null distribution
        (shapiro_test_stat, shapiro_p_val) = scipy.stats.shapiro(samples)
        sample_mean = np.mean(samples)
        sample_std = np.std(samples, ddof=1)
        p_val = scipy.stats.norm.sf(s, loc=sample_mean, scale=sample_std)
        return p_val

    else:
        s = s_XAB(X, s_wAB_memo)
        total_true = 0
        total_equal = 0
        total = 0

        num_partitions = int(scipy.special.binom(2 * len(X), len(X)))
        if num_partitions > n_samples:
            # We only have as much precision as the number of samples drawn;
            # bias the p-value (hallucinate a positive observation) to
            # reflect that.
            total_true += 1
            total += 1
            for _ in range(n_samples - 1):
                np.random.shuffle(XY)
                Xi = XY[:size]
                assert 2 * len(Xi) == len(XY)
                si = s_XAB(Xi, s_wAB_memo)
                if si > s:
                    total_true += 1
                elif si == s:  # use conservative test
                    total_true += 1
                    total_equal += 1
                total += 1
        else:
            for Xi in it.combinations(XY, len(X)):
                Xi = np.array(Xi, dtype=np.int)
                assert 2 * len(Xi) == len(XY)
                si = s_XAB(Xi, s_wAB_memo)
                if si > s:
                    total_true += 1
                elif si == s:  # use conservative test
                    total_true += 1
                    total_equal += 1
                total += 1
        return total_true / total


def mean_s_wAB(X, A, B, cossims):
    return np.mean(s_wAB(A, B, cossims[X]))


def stdev_s_wAB(X, A, B, cossims):
    return np.std(s_wAB(A, B, cossims[X]), ddof=1)


def effect_size(X, Y, A, B, cossims):
    X = list(X)
    Y = list(Y)
    A = list(A)
    B = list(B)

    numerator = mean_s_wAB(X, A, B, cossims=cossims) - mean_s_wAB(Y, A, B, cossims=cossims)
    denominator = stdev_s_wAB(X + Y, A, B, cossims=cossims)
    return numerator / denominator, denominator


def convert_keys_to_ints(X, Y):
    return (
        dict((i, v) for (i, (k, v)) in enumerate(X.items())),
        dict((i + len(X), v) for (i, (k, v)) in enumerate(Y.items())),
    )


def run_test(X,Y,A,B, n_samples, parametric=False):
    # First convert all keys to ints to facilitate array lookups
    (X, Y) = convert_keys_to_ints(X, Y)
    (A, B) = convert_keys_to_ints(A, B)
    XY = X.copy()
    XY.update(Y)
    AB = A.copy()
    AB.update(B)
    cossims = construct_cossim_lookup(XY, AB)
    pval = p_val_permutation_test(X, Y, A, B, n_samples, cossims=cossims, parametric=parametric)
    esize, V_i = effect_size(X, Y, A, B, cossims=cossims)
    return esize, pval, V_i*V_i

In [ ]:
import random
random.seed(42)

def random_ids(length, num=10000):
    xids,yids,aids,bids = [],[],[],[]
    for i in range(num):
        xids.append(random.choice(list(range(length))))
        yids.append(random.choice(list(range(length))))
        aids.append(random.choice(list(range(length))))
        bids.append(random.choice(list(range(length))))
    return xids,yids,aids,bids

xids,yids,aids,bids = random_ids(length=100, num=10000)

In [ ]:
import random

method = 'densray' #hard #conceptor
if method == 'densray':
    import densray_bert as bbert
    path = '/content/drive/My Drive/eigvecs_'+config+'_new_'
elif method == 'hard':
    import hard_bert as bbert
    path = '/content/drive/My Drive/pc1'+config+'_'
elif method == 'conceptor':
    import conceptor_bert as bbert
    path = '/content/drive/My Drive/negc'+config+'_'
#model = bbert.BertForMaskedLM_1.from_pretrained('bert-'+config+'-uncased', eigvecs_dict=get_eigvecs_dict(-1)).to(device)
model = transformers.BertForMaskedLM.from_pretrained('bert-'+config+'-uncased').to(device)
model.eval()

def get_eigvecs_dict(layer=-1):
    eigvecs_dict = {}
    #-1:apply to all layers, -2 nothing, l only on layer lth.
    if layer==-1:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', True)
    elif layer ==-2:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', False)
    else:
        for l in range(nlayer):
            if l==layer:
                eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', True)
            else:
                eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', False)
    return eigvecs_dict


def load_ceat(path):
    f = open(path,'rb')
    ceat = pickle.load(f)
    f.close()
    for word in w_all:
        word_t = tokenizer.convert_tokens_to_ids(word)
        for i in range(len(ceat[word])):
            index = ceat[word][i][0].cpu().detach().tolist().index(word_t)
            #get output embedding
            ceat[word][i] = model.bert(ceat[word][i])[0][0,index,:].cpu().detach().numpy()
    return ceat
ceat = load_ceat('/content/drive/My Drive/ceat_list_100.pkl')

import tqdm
def run_ces(ceat, category, num=10000):
    if category==6:
        X={wd.lower():ceat[wd.lower()] for wd in male}
        Y={wd.lower():ceat[wd.lower()] for wd in female}
        A={wd.lower():ceat[wd.lower()] for wd in career}
        B={wd.lower():ceat[wd.lower()] for wd in family}
    elif category==7:
        X={wd.lower():ceat[wd.lower()] for wd in maths}
        Y={wd.lower():ceat[wd.lower()] for wd in arts}
        A={wd.lower():ceat[wd.lower()] for wd in male_term}
        B={wd.lower():ceat[wd.lower()] for wd in female_term}
    elif category==8:
        X={wd.lower():ceat[wd.lower()] for wd in science}
        Y={wd.lower():ceat[wd.lower()] for wd in arts_8}
        A={wd.lower():ceat[wd.lower()] for wd in male_term_8}
        B={wd.lower():ceat[wd.lower()] for wd in female_term_8}
    
    d_list = []
    p_list = []
    V_list = []
    for i in range(num):
        X = {wd:ceat[wd][xids[i]] for wd in X} #randomly select a vector
        Y = {wd:ceat[wd][yids[i]] for wd in Y} #randomly select a vector
        A = {wd:ceat[wd][aids[i]] for wd in A} #randomly select a vector
        B = {wd:ceat[wd][bids[i]] for wd in X} #randomly select a vector
        d, p, V_i = run_test(X,Y,A,B, 1000, True)
        d_list.append(d)
        p_list.append(p)
        V_list.append(V_i)
    d_list, p_list, V_list = np.array(d_list), np.array(p_list), np.array(V_list)
    W_list = 1/V_list
    c = W_list.sum() - (W_list*W_list).sum()/W_list.sum()
    Q = (W_list*(d_list*d_list)).sum() - (W_list*d_list).sum()/W_list.sum()
    sigma = (Q-(num-1))/c if Q>=(num-1) else 0
    v_i = 1/(V_list+sigma)
    CES = (v_i*p_list).sum()/v_i.sum()
    
    pses = np.mean((p_list < 0.05))
    pes = np.mean(d_list)
    return pes, pses, CES

for c in [6,7,8]:
    pes, pses, CES = run_ces(ceat, c)
    print(pes, pses, CES, '\n')

-1.0308359870264727 0.0001 0.942160388726403 

-1.3082747113357986 0.0 0.993377609583838 

-1.5655638737829822 0.0 0.9985365274583365 



In [ ]:
import random

method = 'densray' #hard #conceptor
if method == 'densray':
    import densray_bert as bbert
    path = '/content/drive/My Drive/eigvecs_'+config+'_new_'
elif method == 'hard':
    import hard_bert as bbert
    path = '/content/drive/My Drive/pc1'+config+'_'
elif method == 'conceptor':
    import conceptor_bert as bbert
    path = '/content/drive/My Drive/negc'+config+'_'
model = bbert.BertForMaskedLM_1.from_pretrained('bert-'+config+'-uncased', eigvecs_dict=get_eigvecs_dict(-1)).to(device)
#model = transformers.BertForMaskedLM.from_pretrained('bert-'+config+'-uncased').to(device)
model.eval()

def get_eigvecs_dict(layer=-1):
    eigvecs_dict = {}
    #-1:apply to all layers, -2 nothing, l only on layer lth.
    if layer==-1:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', True)
    elif layer ==-2:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', False)
    else:
        for l in range(nlayer):
            if l==layer:
                eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', True)
            else:
                eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', False)
    return eigvecs_dict


def load_ceat(path):
    f = open(path,'rb')
    ceat = pickle.load(f)
    f.close()
    for word in w_all:
        word_t = tokenizer.convert_tokens_to_ids(word)
        for i in range(len(ceat[word])):
            index = ceat[word][i][0].cpu().detach().tolist().index(word_t)
            #get output embedding
            ceat[word][i] = model.bert(ceat[word][i])[0][0,index,:].cpu().detach().numpy()
    return ceat
ceat = load_ceat('/content/drive/My Drive/ceat_list_100.pkl')

import tqdm
def run_ces(ceat, category, num=10000):
    if category==6:
        X={wd.lower():ceat[wd.lower()] for wd in male}
        Y={wd.lower():ceat[wd.lower()] for wd in female}
        A={wd.lower():ceat[wd.lower()] for wd in career}
        B={wd.lower():ceat[wd.lower()] for wd in family}
    elif category==7:
        X={wd.lower():ceat[wd.lower()] for wd in maths}
        Y={wd.lower():ceat[wd.lower()] for wd in arts}
        A={wd.lower():ceat[wd.lower()] for wd in male_term}
        B={wd.lower():ceat[wd.lower()] for wd in female_term}
    elif category==8:
        X={wd.lower():ceat[wd.lower()] for wd in science}
        Y={wd.lower():ceat[wd.lower()] for wd in arts_8}
        A={wd.lower():ceat[wd.lower()] for wd in male_term_8}
        B={wd.lower():ceat[wd.lower()] for wd in female_term_8}
    
    d_list = []
    p_list = []
    V_list = []
    for i in range(num):
        X = {wd:ceat[wd][xids[i]] for wd in X} #randomly select a vector
        Y = {wd:ceat[wd][yids[i]] for wd in Y} #randomly select a vector
        A = {wd:ceat[wd][aids[i]] for wd in A} #randomly select a vector
        B = {wd:ceat[wd][bids[i]] for wd in X} #randomly select a vector
        d, p, V_i = run_test(X,Y,A,B, 1000, True)
        d_list.append(d)
        p_list.append(p)
        V_list.append(V_i)
    d_list, p_list, V_list = np.array(d_list), np.array(p_list), np.array(V_list)
    W_list = 1/V_list
    c = W_list.sum() - (W_list*W_list).sum()/W_list.sum()
    Q = (W_list*(d_list*d_list)).sum() - (W_list*d_list).sum()/W_list.sum()
    sigma = (Q-(num-1))/c if Q>=(num-1) else 0
    v_i = 1/(V_list+sigma)
    CES = (v_i*p_list).sum()/v_i.sum()
    
    pses = np.mean((p_list < 0.05))
    pes = np.mean(d_list)
    return pes, pses, CES

for c in [6,7,8]:
    pes, pses, CES = run_ces(ceat, c)
    print(pes, pses, CES, '\n')

-0.4710406170037398 0.0037 0.7522627975222524 

-1.3486192547018008 0.0 0.9947924763352177 

-1.5559406969294876 0.0 0.9984912656334195 



In [ ]:
import random

method = 'hard' #hard #conceptor
if method == 'densray':
    import densray_bert as bbert
    path = '/content/drive/My Drive/eigvecs_'+config+'_new_'
elif method == 'hard':
    import hard_bert as bbert
    path = '/content/drive/My Drive/pc1'+config+'_'
elif method == 'conceptor':
    import conceptor_bert as bbert
    path = '/content/drive/My Drive/negc'+config+'_'
model = bbert.BertForMaskedLM_1.from_pretrained('bert-'+config+'-uncased', eigvecs_dict=get_eigvecs_dict(-1)).to(device)
#model = transformers.BertForMaskedLM.from_pretrained('bert-'+config+'-uncased').to(device)
model.eval()

def get_eigvecs_dict(layer=-1):
    eigvecs_dict = {}
    #-1:apply to all layers, -2 nothing, l only on layer lth.
    if layer==-1:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', True)
    elif layer ==-2:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', False)
    else:
        for l in range(nlayer):
            if l==layer:
                eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', True)
            else:
                eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', False)
    return eigvecs_dict


def load_ceat(path):
    f = open(path,'rb')
    ceat = pickle.load(f)
    f.close()
    for word in w_all:
        word_t = tokenizer.convert_tokens_to_ids(word)
        for i in range(len(ceat[word])):
            index = ceat[word][i][0].cpu().detach().tolist().index(word_t)
            #get output embedding
            ceat[word][i] = model.bert(ceat[word][i])[0][0,index,:].cpu().detach().numpy()
    return ceat
ceat = load_ceat('/content/drive/My Drive/ceat_list_100.pkl')

import tqdm
def run_ces(ceat, category, num=10000):
    if category==6:
        X={wd.lower():ceat[wd.lower()] for wd in male}
        Y={wd.lower():ceat[wd.lower()] for wd in female}
        A={wd.lower():ceat[wd.lower()] for wd in career}
        B={wd.lower():ceat[wd.lower()] for wd in family}
    elif category==7:
        X={wd.lower():ceat[wd.lower()] for wd in maths}
        Y={wd.lower():ceat[wd.lower()] for wd in arts}
        A={wd.lower():ceat[wd.lower()] for wd in male_term}
        B={wd.lower():ceat[wd.lower()] for wd in female_term}
    elif category==8:
        X={wd.lower():ceat[wd.lower()] for wd in science}
        Y={wd.lower():ceat[wd.lower()] for wd in arts_8}
        A={wd.lower():ceat[wd.lower()] for wd in male_term_8}
        B={wd.lower():ceat[wd.lower()] for wd in female_term_8}
    
    d_list = []
    p_list = []
    V_list = []
    for i in range(num):
        X = {wd:ceat[wd][xids[i]] for wd in X} #randomly select a vector
        Y = {wd:ceat[wd][yids[i]] for wd in Y} #randomly select a vector
        A = {wd:ceat[wd][aids[i]] for wd in A} #randomly select a vector
        B = {wd:ceat[wd][bids[i]] for wd in X} #randomly select a vector
        d, p, V_i = run_test(X,Y,A,B, 1000, True)
        d_list.append(d)
        p_list.append(p)
        V_list.append(V_i)
    d_list, p_list, V_list = np.array(d_list), np.array(p_list), np.array(V_list)
    W_list = 1/V_list
    c = W_list.sum() - (W_list*W_list).sum()/W_list.sum()
    Q = (W_list*(d_list*d_list)).sum() - (W_list*d_list).sum()/W_list.sum()
    sigma = (Q-(num-1))/c if Q>=(num-1) else 0
    v_i = 1/(V_list+sigma)
    CES = (v_i*p_list).sum()/v_i.sum()
    
    pses = np.mean((p_list < 0.05))
    pes = np.mean(d_list)
    return pes, pses, CES

for c in [6,7,8]:
    pes, pses, CES = run_ces(ceat, c)
    print(pes, pses, CES, '\n')

-0.7989600833063635 0.001 0.8754349346581141 

-1.3528777585733467 0.0 0.9949020566450059 

-1.5780045754684953 0.0 0.9986603470779494 



In [ ]:
import random

method = 'conceptor' #hard #conceptor
if method == 'densray':
    import densray_bert as bbert
    path = '/content/drive/My Drive/eigvecs_'+config+'_new_'
elif method == 'hard':
    import hard_bert as bbert
    path = '/content/drive/My Drive/pc1'+config+'_'
elif method == 'conceptor':
    import conceptor_bert as bbert
    path = '/content/drive/My Drive/negc'+config+'_'
model = bbert.BertForMaskedLM_1.from_pretrained('bert-'+config+'-uncased', eigvecs_dict=get_eigvecs_dict(11)).to(device)
#model = transformers.BertForMaskedLM.from_pretrained('bert-'+config+'-uncased').to(device)
model.eval()

def get_eigvecs_dict(layer=-1):
    eigvecs_dict = {}
    #-1:apply to all layers, -2 nothing, l only on layer lth.
    if layer==-1:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', True)
    elif layer ==-2:
        for l in range(nlayer):
            eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', False)
    else:
        for l in range(nlayer):
            if l==layer:
                eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', True)
            else:
                eigvecs_dict[str(l)] = (path+str(nsamples)+'_'+str(l)+'.pt', False)
    return eigvecs_dict


def load_ceat(path):
    f = open(path,'rb')
    ceat = pickle.load(f)
    f.close()
    for word in w_all:
        word_t = tokenizer.convert_tokens_to_ids(word)
        for i in range(len(ceat[word])):
            index = ceat[word][i][0].cpu().detach().tolist().index(word_t)
            #get output embedding
            ceat[word][i] = model.bert(ceat[word][i])[0][0,index,:].cpu().detach().numpy()
    return ceat
ceat = load_ceat('/content/drive/My Drive/ceat_list_100.pkl')

import tqdm
def run_ces(ceat, category, num=10000):
    if category==6:
        X={wd.lower():ceat[wd.lower()] for wd in male}
        Y={wd.lower():ceat[wd.lower()] for wd in female}
        A={wd.lower():ceat[wd.lower()] for wd in career}
        B={wd.lower():ceat[wd.lower()] for wd in family}
    elif category==7:
        X={wd.lower():ceat[wd.lower()] for wd in maths}
        Y={wd.lower():ceat[wd.lower()] for wd in arts}
        A={wd.lower():ceat[wd.lower()] for wd in male_term}
        B={wd.lower():ceat[wd.lower()] for wd in female_term}
    elif category==8:
        X={wd.lower():ceat[wd.lower()] for wd in science}
        Y={wd.lower():ceat[wd.lower()] for wd in arts_8}
        A={wd.lower():ceat[wd.lower()] for wd in male_term_8}
        B={wd.lower():ceat[wd.lower()] for wd in female_term_8}
    
    d_list = []
    p_list = []
    V_list = []
    for i in range(num):
        X = {wd:ceat[wd][xids[i]] for wd in X} #randomly select a vector
        Y = {wd:ceat[wd][yids[i]] for wd in Y} #randomly select a vector
        A = {wd:ceat[wd][aids[i]] for wd in A} #randomly select a vector
        B = {wd:ceat[wd][bids[i]] for wd in X} #randomly select a vector
        d, p, V_i = run_test(X,Y,A,B, 1000, True)
        d_list.append(d)
        p_list.append(p)
        V_list.append(V_i)
    d_list, p_list, V_list = np.array(d_list), np.array(p_list), np.array(V_list)
    W_list = 1/V_list
    c = W_list.sum() - (W_list*W_list).sum()/W_list.sum()
    Q = (W_list*(d_list*d_list)).sum() - (W_list*d_list).sum()/W_list.sum()
    sigma = (Q-(num-1))/c if Q>=(num-1) else 0
    v_i = 1/(V_list+sigma)
    CES = (v_i*p_list).sum()/v_i.sum()
    
    pses = np.mean((p_list < 0.05))
    pes = np.mean(d_list)
    return pes, pses, CES

for c in [6,7,8]:
    pes, pses, CES = run_ces(ceat, c)
    print(pes, pses, CES, '\n')

-1.0035016495250082 0.0 0.944417453453168 

-1.3712686272949601 0.0 0.9955850835976079 

-1.6322733492374912 0.0 0.9991663835833512 



In [ ]:
for_ceat('/content/drive/My Drive/wikien_senttok.txt','/content/drive/My Drive/ceat_base_hard.pkl')

In [ ]:
1

1